In [1]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import cohere

co = cohere.Client('VLLVyGHAXjiYz8frehzDEFPgjF1BkLf05P2RXlQx')

In [3]:
knowledge_base = {
    "conditions": {
        "pcos": {
            "symptoms": [
                "Irregular menstrual cycles",
                "Excessive hair growth",
                "Acne",
                "Obesity",
                "Ovarian cysts"
            ],
            "treatment": [
                "Lifestyle changes (diet, exercise)",
                "Medications (metformin, hormonal contraceptives)",
                "Fertility treatments if necessary"
            ],
            "diagnostic_criteria": [
                "Irregular periods",
                "Elevated androgen levels",
                "Polycystic ovaries on ultrasound"
            ],
            "complications": [
                "Infertility",
                "Type 2 diabetes",
                "Heart disease",
                "Endometrial cancer"
            ]
        },
        "endometriosis": {
            "symptoms": [
                "Pelvic pain",
                "Painful periods",
                "Pain during intercourse",
                "Infertility",
                "Heavy menstrual bleeding"
            ],
            "treatment": [
                "Pain relief medications (NSAIDs)",
                "Hormonal therapies (birth control pills, GnRH agonists)",
                "Surgical options (laparoscopy, hysterectomy)"
            ],
            "diagnostic_criteria": [
                "Pelvic exam",
                "Ultrasound",
                "MRI",
                "Laparoscopy"
            ],
            "complications": [
                "Chronic pain",
                "Ovarian cysts",
                "Scar tissue and adhesions",
                "Infertility"
            ]
        },
        "uterine_fibroids": {
            "symptoms": [
                "Heavy menstrual bleeding",
                "Pelvic pressure or pain",
                "Frequent urination",
                "Difficulty emptying the bladder",
                "Constipation"
            ],
            "treatment": [
                "Medications (hormone therapy)",
                "Minimally invasive procedures (uterine artery embolization)",
                "Surgical options (myomectomy, hysterectomy)"
            ],
            "diagnostic_criteria": [
                "Pelvic exam",
                "Ultrasound",
                "MRI"
            ],
            "complications": [
                "Anemia from heavy blood loss",
                "Urinary tract infections",
                "Infertility"
            ]
        },
        "adenomyosis": {
            "symptoms": [
                "Heavy or prolonged menstrual bleeding",
                "Severe menstrual cramps",
                "Chronic pelvic pain",
                "Enlarged uterus"
            ],
            "treatment": [
                "Pain relief medications (NSAIDs)",
                "Hormone therapy (birth control pills, IUD)",
                "Surgical options (endometrial ablation, hysterectomy)"
            ],
            "diagnostic_criteria": [
                "Pelvic exam",
                "Ultrasound",
                "MRI"
            ],
            "complications": [
                "Severe pain",
                "Anemia from heavy blood loss",
                "Impact on quality of life"
            ]
        }
    }
}

import json

# Save the knowledge base to a JSON file
with open('knowledge_base.json', 'w') as f:
    json.dump(knowledge_base, f)

In [5]:
import json
import re
import cohere

class ClinicalDecisionSupportBot:
    def __init__(self, api_key, knowledge_base_path):
        self.cohere_client = cohere.Client(api_key)
        self.knowledge_base = self.load_knowledge_base(knowledge_base_path)

    def load_knowledge_base(self, path):
        with open(path, 'r') as file:
            return json.load(file)

    def preprocess_query(self, query):
        query = query.lower()
        query = re.sub(r'[^\w\s]', '', query)
        return query

    def generate_response(self, prompt):
        response = self.cohere_client.generate(
            model='large',
            prompt=prompt,
            max_tokens=100
        )
        return response.generations[0].text

    def answer_query(self, query):
        preprocessed_query = self.preprocess_query(query)
        for condition, details in self.knowledge_base["conditions"].items():
            if condition in preprocessed_query:
                if "symptoms" in preprocessed_query:
                    return f"The common symptoms of {condition.upper()} include: {', '.join(details['symptoms'])}."
                elif "treatment" in preprocessed_query:
                    return f"The recommended treatment for {condition.upper()} includes: {', '.join(details['treatment'])}."
                elif "diagnostic criteria" in preprocessed_query:
                    return f"The diagnostic criteria for {condition.upper()} include: {', '.join(details['diagnostic_criteria'])}."
                elif "complications" in preprocessed_query:
                    return f"{condition.upper()} can lead to the following complications: {', '.join(details['complications'])}."

        prompt = f"Provide clinical decision support for: {query}"
        response = self.generate_response(prompt)
        return response

# Initialize the bot with your API key and knowledge base file path
bot = ClinicalDecisionSupportBot('VLLVyGHAXjiYz8frehzDEFPgjF1BkLf05P2RXlQx', 'knowledge_base.json')

# Test the bot with a query
print(bot.answer_query("What are the symptoms of PCOS?"))
print(bot.answer_query("What are the treatment options for endometriosis?"))

The common symptoms of PCOS include: Irregular menstrual cycles, Excessive hair growth, Acne, Obesity, Ovarian cysts.
The recommended treatment for ENDOMETRIOSIS includes: Pain relief medications (NSAIDs), Hormonal therapies (birth control pills, GnRH agonists), Surgical options (laparoscopy, hysterectomy).


In [7]:
# Interactive loop for the chatbot
while True:
    user_query = input("Enter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    response = bot.answer_query(user_query)
    print("Chatbot:", response)

Enter your query (or type 'exit' to quit): What are the symptoms of PCOS?
Chatbot: The common symptoms of PCOS include: Irregular menstrual cycles, Excessive hair growth, Acne, Obesity, Ovarian cysts.
Enter your query (or type 'exit' to quit): What are the treatment options for endometriosis?
Chatbot: The recommended treatment for ENDOMETRIOSIS includes: Pain relief medications (NSAIDs), Hormonal therapies (birth control pills, GnRH agonists), Surgical options (laparoscopy, hysterectomy).
Enter your query (or type 'exit' to quit): exit
